# Generación de variables aleatorias discretas

## Método de la Transformada Inversa

Dado $X$ una variable aleatoria discreta con función de probabilidad $P(X=x_j)=p_j$, y función de distribución acumulada

$F_X(x) = \begin{cases}
    0 & \text{si } x<x_0\\
    p_0+...+p_{n-1} & \text{cc}\\
\end{cases}$,

para $j=0,1,...$, $n=1,2,...$. Luego generamos $U\thicksim\mathcal{U}(0,1)$ y definimos

$X = \begin{cases}
    x_0 & \text{si } U\in [0,p_0)\\
    x_n & \text{si } U\in [p_0+...+p_{n-1},p_o+...+p_n)\\
\end{cases}$

In [1]:
from random import random

def MetodoTransformadaInversa(p, x):
    U=random()
    i=0; F=p[i]
    while U >= F:
        i += 1; F += p[i]
    return x[i]

### Cálculo de promedios
Supongamos que se desea calcular un promedio de una gran cantidad de valores:
$\bar{a}= \frac{1}{N}\sum_{i=1}^{N}a_i = \frac{a_1+...+a_N}{N}.$
donde $N>>1$ y por la gran cantidad de términos o por la complejidad de éstos resulta muy complicado calcular la suma.
Si definimos $X$ una variable aleatoria uniforme discreta en $[1,N]$, y $g$ una función tal que $g(i)=a_i$, entonces el valor que se desea calcular es $\bar{a}=E[g(X)]$.

In [ ]:
from numpy import exp

def Promedio():
    Suma=0
    Nsim=100
    for _ in range(Nsim):
        U = int(random() * 10000) + 1
        Suma += exp(1 / U)
    Suma = Suma / Nsim * 10000

### Generación de una variable aleatoria uniforme discreta
Si $X$ es una variable con distribución uniforme discreta en $\{1,...,n\}$ entonces $p_1=...=p_n=1/n$, luego la función de distribución acumulada es

$F_X(x) = \begin{cases}
    0 & \text{si } x<1\\
    1/n+...+1/n & \text{cc}\\
\end{cases}$

Aplicando el método de la inversa,

$X$
genera
$
j \Leftrightarrow \\
p_0+...+p_{j-1} \le U < p_o+...+p_j \Leftrightarrow \\
(j-1)/n \le U<j/n \Leftrightarrow \\
j-1\le Un<j \Leftrightarrow \\
j-1=\lfloor Un \rfloor\Leftrightarrow \\
j=\lfloor Un \rfloor +1
$


In [ ]:
def VariableUniformeDiscreta(n):
    return int(n*random()) + 1

### Generación de una variable aleatoria geométrica
Si $X$ es una variable con distribución geométrica con probabilidad de éxito $p$ tiene una probabilidad de masa dada por $p_i=P(X=i)=pq^{i-1}, \quad i\ge1, \quad q=(1-p),$ y función de distribución acumulada $F_X(j-1)=P(X\le j-1)=1-P(X > j-1)=1-q^{j-1}$.

Aplicando el método de la inversa,

$X$
genera
$
j \Leftrightarrow \\
p_0+...+p_{j-1} \le U < p_o+...+p_j \Leftrightarrow \\
1-q^{j-1} \le U < 1-q^j \Leftrightarrow \\
-q^{j-1} \le U-1 < -q^j \Leftrightarrow \\
q^j < 1-U \le q^{j-1} \Leftrightarrow \\
jLog(q) <  Log(1-U) \le (j-1)Log(q) \Leftrightarrow \\
j <  \frac{Log(1-U)}{Log(q)} \le j-1 \Leftrightarrow \quad \{ q<1 \Rightarrow Log(q)<1 \} \\
j-1 \le  \frac{Log(1-U)}{Log(q)} < j \Leftrightarrow \\
j-1 = \lfloor \frac{Log(1-U)}{Log(q)} \rfloor \Leftrightarrow \\
j = \lfloor \frac{Log(1-U)}{Log(q)} \rfloor +1 \\
$



In [ ]:
from numpy import log

def VariableAleatoriaGeometrica(p):
    return (int(log(1-random())/log(1-p)) + 1)

### Generación de una variable aleatoria Bernoulli

In [ ]:
def VariableAleatoriaBernoulli(p):
    U=random()
    if U < p: return 1
    else:     return 0

### Generación de una variable aleatoria Poisson
La función de probabilidad de masa de una variable aleatoria Poisson de razón $\lambda$ está dada por
$\\ p_i=P(X=i)=e^{-\lambda}\frac{\lambda^i}{i!}, \quad i=0,1,...\\$
Luego, podemos notar que las probabilidades cumplen una relación de recurrencia:
$\\p_0 = e^{-\lambda}, \quad p_i = \frac{\lambda}{i}p_{i-1}, \quad i=1,2,...$


In [ ]:
def VariableAleatoriaPoisson(lamda):
    U = random()
    i = 0; p = exp(-lamda)
    F = p
    while U >= F:
        i += 1
        p += lamda / i
        F = F + p
    return i

Una forma de optimizar este algoritmo es comenzar por analizar el valor mas probable. El valor máximo de las probabilidades es $p_{\lfloor λ\rfloor}$, es decir, que los valores cercanos a λ son los mas probables. Así, se puede optimizar el número de comparaciones buscando a partir del valor $I = \lfloor λ\rfloor$, y luego de manera ascendente o descendente según el valor de la variable uniforme U generada:

In [ ]:
def Poisson(lamda):
    p = exp(-lamda); F = p
    for j in range(1, int(lamda) + 1):
        p *= lamda / j
        F += p
    U = random()
    if U >= F:
        j = int(lamda) + 1
        while U >= F:
            p *= lamda / j; F += p
            j += 1
        return j - 1
    else:
        j = int(lamda)
        while U < F:
            F -= p; p *= j/lamda
            j -= 1
        return j+1

De esta manera, el promedio de búsquedas se reduce a $1+E[|X-\lambda|]$. En particular, si $\lambda >> 1$, la distrubución de Poisson se aproxima a una normal con media y varianza $\lambda$, por lo cual $\frac{X-\lambda}{\sqrt\lambda}$ se aproxima a una normal estándar, y por lo tanto, $1+E[|X-\lambda|]$, puede estimarse como:
$\\1+\sqrt{\lambda}E[\frac{|X-\lambda|}{\sqrt{\lambda}}]=1+\sqrt{\lambda}E[|Z|]=1+0.798\sqrt{\lambda}\\$
que mejora el promedio en la versión anterior $1+\lambda$.

### Generación de una variable aleatoria binomial
Si $X\thicksim B(n,p)$ es una variable con distribución binomial con función de masa de probabilidades 
$\\p_i=P(X=i)=\frac{n!}{i!(n-1)!}p^i(1-p)^{n-i}, \quad i=0,1,...,n.\\$
En este caso, la fórmula recursiva para las probabilidades está dada por:
$\\ p_0=(1-p)^n, \quad p_{i+1}=\frac{n-i}{i+1}\frac{p}{1-p}p_i, \quad 0\le i<n\\$
Si se aplica directamente el método de la transformada inversa, el algoritmo resulta:

In [ ]:
def VariableAleatoriaBinomial(n,p):
    c = p / (1 - p)
    prob = (1 - p) ** n
    F = prob; i=0
    U = random()
    while U >= F:
        prob *= c * (n-i) / (i+1)
        F += prob
        i += 1
    return i    

## Método de aceptación y rechazo (método de rechazo)
Supone el conocimiento de un método para la generación de otra variable aleatoria $Y$ que cumpla con las siguientes propiedades:
* Si $P(X=x_j)>0$, entonces $P(Y=x_j)>0$, para todo $X_j$ en el rango de $X$.
* Existe una constante $s$ tal que $\frac{P(X=x_j)}{P(Y=y_j)}\le c$, para todos los $x_j$ tal que $P(X=x_j)>0$.

Si denotamos $p_j = P(X=x_j)$ y $q_j = P(Y=x_j)$, entonces de la segunda propoiedad vemos que:
$\\\sum_{j\ge1}p_j\le c\times\sum_{j\ge1}q_j\le c.\\$

Como el miembro izquierdo de la desigualdad es 1, se tiene que la constante $c$ es siempre
mayor o igual a 1. Además la igualdad se daría solo en el caso en que $X$ e $Y$ tienen la misma
distribución de probabilidad, en cuyo caso ya se tendría un método para generar $X$. Asumimos
entonces que $c > 1$, y por lo tanto $\frac1c < 1$.

In [ ]:
def MetodoAceptacionRechazo(SimularY,p,q,c):
    Y, U = SimularY(), random()
    while U >= p(Y) / (c * q(Y)):
        Y, U = SimularY(), random() # rechazo
    return Y # aceptación

## Método de composición
El método de composición permite generar una variable aleatoria $X$ con función de probabilidad de masa dada por: $\\P(X=j)=\alpha p_j+(1-\alpha)q_j, \quad j=0,1,...,\\$ donde $\alpha$ es algún número entre 0 y 1. Entonces, si se tienen métodos eficientes para generar valores de dos variables aleatorias $X_1$ y $X_2$ con funciones de probabilidad de masa $p_j$ y $q_j$ respectivamente, entonces se puede generar $X$ de la siguiente manera:
$\\X = \begin{cases}
    X_1 & \text{con probabilidad } \alpha\\
    X_2 & \text{con probabilidad } 1-\alpha\\
\end{cases}$

In [ ]:
def MetodoComposicion(alpha,SimularX1,simularX2):
    U = random()
    if U < alpha:
        return SimularX1()
    else:
        return simularX2()


## Método del alias
Es una aplicación del método de composición para generar variables aleatorias discretas que toman los valores $1,2,...n.$ Requiere construir en primer lugar los llamados *alias*, pero una vez completada este etapa, la simulación en sí es muy simple. La distribución de la variable X, F_X, se describe como una composición equiprobable de $n-1$ variables aleatorias:
$\\F_X(x)=\frac1{n-1}F_1(x)+...+\frac1{n-1}F_{n-1}(x),\\$
y $F_j(x)$ es la distribución de una variable $X_j$, que toma a lo sumo dos valores posibles.

**Proposición:** Sea $P(X=j)=p_j,1\le j\le n$ una función de probabilidad de masa, con $n>1$. Entonces:
1. Existe $j$, $1\le j\le n$, tal que $p_j<\frac1{n-1}$.
2. Dado $j$ como en 1, existe $i \ne j$ tal que $p_i+p_j\ge\frac1{n-1}$.

    **Demostración:** Supongamos que no existe tal $j$. Entonces, $\sum_{j=1}^np_j\ge n\frac1{n-1}>1$, lo cual es una contradicción. Por otro lado, dado $j$ como en 1, debe existir $i$ que cumpla la segunda propiedad, ya que si no existiera, entonces $1-p_j=\sum_{i\ne j}p_i<(n-1)(\frac1{n-1}-p_j)=1-(n-1)p_j$. Esto es, $p_j>(n-1)-p_j$, lo cual solo es valido si $n=1$.

El procedimiento es como sigue. En primer lugar, notemos que si se multiplican todas las probabilidades por $n − 1$, entonces su suma es igual a $n − 1$:
$\\(n − 1)p1, (n − 1)p2, , . . . , (n − 1)pn. (*)\\$
La idea es distribuir estos $n − 1$ valores dados como probabilidades de masa de $n − 1$
variables aleatorias, de modo que estas variables tomen a lo sumo dos valores distintos.
Si se tuviera una variable $X$ que toma valores en $\{1, 2, . . . , n\}$ con probabilidad de masa $p1, p2, . . . , pn$, entonces se eligen $j$ e $i$ como en la proposición, y se define $X$ de la siguiente manera:
$\\ X_1 = \begin{cases}
    j & \text{con probabilidad } (n-1)p_j\\
    i & \text{con probabilidad } 1-(n-1)p_j\\
\end{cases}$

Notemos que $X_1$ toma todo el peso del valor $j$, y parte del peso del valor $i$. Digamos que de
la lista (*) de valores se han utilizado $p_j$ y una ”parte” de $p_i$, de modo que suman 1. Si se
restan estos dos valores de la lista(*), la suma total es igual a $n − 2$.
Es decir, si dividimos por $n − 2$ se tendría nuevamente una variable aleatoria $\bar{X}$ con valores en $\{1, 2, . . . , n\} − \{j\}$, que tiene probabilidades
$\\P(\bar{X}=k)=\frac1{n-2}(n-1)p_k, \quad k\ne j,i.\\$
y $P(\bar{X}=i)$ es tal que la suma de todas las probabilidades es 1.
Ahora se eligen nuevos índices $j$ e $i$ con las propiedades dadas por la Proposición, y se construye una variable $X_2$, cambiando $n$ por $n − 1$. El procedimiento sigue hasta que solo quedan a lo sumo dos valores por generar.


In [ ]:
'''
Ejemplo de X una v.a. que toma valores en {1,2,3,4} con
p1=7/16, p2=1/4, p3=1/8, p4=3/16
'''
def MetodoAlias():
    I = int (random() * 3)
    V = random()
    if I == 0:
        if V < 5/8: return 1
        else: return 3
    elif I == 1:
        if V < 9/16: return 4
        else: return 2
    else: ##I == 2
        if V < 11/16: return 1
        else: return 2

## Método de la urna
Dada una variable aleatoria $X$, que toma un numero finito de valores, digamos $\{1, 2, . . . , n\}$, llamamos $p_j = P(X = j)$. El método consiste en considerar un valor $k ∈ \N$ tal
que $k p_j$ sea entero, para todo $j \in \{1, n\}$.
Ahora se considera un arreglo $A$ de $k$ posiciones, y se almacena cada valor $i$ en $k p_i$ posiciones del arreglo.
El algoritmo simplemente selecciona una posición al azar del arreglo y devuelve el valor en
dicha posición.

In [ ]:
def MetodoUrna(A,k):
    I = int(random() * k)
    return A[I]